Important questions:
* How to test this as a stream machine learning approach? (online learning)
    * Could I simply hold out one or two of the types of attacks and then "add it in"? (check literature on this part)

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, IsolationForest

In [3]:
df = pd.read_csv("../data/raw/KDDTrain+.txt", header=None)

In [4]:
df

,0,1,2,3,4,5,6,7,8,9,...,33,34,35,36,37,38,39,40,41,42
0,0,tcp,ftp_data,SF,491,0,0,0,0,0,...,0.17,0.03,0.17,0.00,0.00,0.00,0.05,0.00,normal,20
1,0,udp,other,SF,146,0,0,0,0,0,...,0.00,0.60,0.88,0.00,0.00,0.00,0.00,0.00,normal,15
2,0,tcp,private,S0,0,0,0,0,0,0,...,0.10,0.05,0.00,0.00,1.00,1.00,0.00,0.00,neptune,19
3,0,tcp,http,SF,232,8153,0,0,0,0,...,1.00,0.00,0.03,0.04,0.03,0.01,0.00,0.01,normal,21
4,0,tcp,http,SF,199,420,0,0,0,0,...,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,normal,21
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
125968,0,tcp,private,S0,0,0,0,0,0,0,...,0.10,0.06,0.00,0.00,1.00,1.00,0.00,0.00,neptune,20
125969,8,udp,private,SF,105,145,0,0,0,0,...,0.96,0.01,0.01,0.00,0.00,0.00,0.00,0.00,normal,21
125970,0,tcp,smtp,SF,2231,384,0,0,0,0,...,0.12,0.06,0.00,0.00,0.72,0.00,0.01,0.00,normal,18
125971,0,tcp,klogin,S0,0,0,0,0,0,0,...,0.03,0.05,0.00,0.00,1.00,1.00,0.00,0.00,neptune,20


In [5]:
df[41].value_counts()

normal             67343
neptune            41214
satan               3633
ipsweep             3599
portsweep           2931
smurf               2646
nmap                1493
back                 956
teardrop             892
warezclient          890
pod                  201
guess_passwd          53
buffer_overflow       30
warezmaster           20
land                  18
imap                  11
rootkit               10
loadmodule             9
ftp_write              8
multihop               7
phf                    4
perl                   3
spy                    2
Name: 41, dtype: int64

In [6]:
pd.get_dummies(df[3])

,OTH,REJ,RSTO,RSTOS0,RSTR,S0,S1,S2,S3,SF,SH
0,0,0,0,0,0,0,0,0,0,1,0
1,0,0,0,0,0,0,0,0,0,1,0
2,0,0,0,0,0,1,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,1,0
4,0,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...
125968,0,0,0,0,0,1,0,0,0,0,0
125969,0,0,0,0,0,0,0,0,0,1,0
125970,0,0,0,0,0,0,0,0,0,1,0
125971,0,0,0,0,0,1,0,0,0,0,0


In [7]:
df = pd.concat([df, pd.get_dummies(df[1]), pd.get_dummies(df[2]), pd.get_dummies(df[3])], axis=1)

In [8]:
def convert_target(x):
    if x == "normal":
        return 0
    else:
        return 1

better_df = df.drop([1,2,3], axis=1)
better_df[41] = better_df[41].apply(lambda x: convert_target(x))

better_df.to_csv("../data/cache/cleaned.csv")

X = df.drop([1, 2, 3, 41], axis=1)
y = df[41].apply(lambda x: convert_target(x))

In [ ]:
better_df.columns

In [10]:
X.shape

(125973, 123)

In [11]:
df[40].shape

(125973,)

In [12]:
X_train, X_test, y_train, y_rest = train_test_split(X, y, test_size=.2, random_state=13)

# Logistic Regression

In [13]:
clf = LogisticRegression(max_iter=1000).fit(X_train, y_train)

/usr/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [14]:
clf.score(X_test, y_rest)

0.8865647946021036

# Random Forest

In [15]:
forest = RandomForestClassifier(200, "entropy", verbose=1)
forest.fit(X_train, y_train)
forest.score(X_test, y_rest)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 200 out of 200 | elapsed:   15.9s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 200 out of 200 | elapsed:    0.5s finished


0.9996824766818814

# Isolation Forest

In [ ]:
iso_forest = IsolationForest(200, verbose=1)
iso_forest.fit(X_train, y_train)
iso_forest.score_samples(X_test)

# As Stream Learning

In [ ]:
pd.read_csv("../data/cache/cleaned.csv")

In [3]:
%matplotlib qt

In [4]:
from skmultiflow.data import FileStream
from skmultiflow.trees import HoeffdingTree
from skmultiflow.evaluation import EvaluatePrequential

In [5]:
stream = FileStream("../data/cache/cleaned.csv")
stream.prepare_for_use()

In [7]:
ht = HoeffdingTree()

In [ ]:
evaluator = EvaluatePrequential(pretrain_size=1000, max_samples=10000, show_plot=True)
evaluator.evaluate(stream=stream, model=ht)

Prequential Evaluation
Evaluating 1 target(s).
Pre-training on 1000 sample(s).
Evaluating...
 #################### [100%] [18.98s]
Processed samples: 10000
Mean performance:
M0 - Accuracy     : 0.9988
M0 - Kappa        : 0.5213


## Resources

https://towardsdatascience.com/ensemble-learning-using-scikit-learn-85c4531ff86a

